In [1]:
import warnings
warnings.filterwarnings("ignore")

# loading packages

import numpy as np
import pandas as pd
from pandas import datetime
from pandas import Series,DataFrame
import matplotlib.pyplot as plt


from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error


import logging

import dvc.api

In [2]:
Format = '%(levelname)s %(asctime)s - %(message)s'
logging.basicConfig(filename= r'C:\Users\Jakinda\Documents\Python Scripts\10Academy\PharmaceuticalSalesPrediction\Notebook\ml.log',
level =logging.INFO,
format = Format)
logger = logging.getLogger()

In [3]:
#Get url from DVC

path = r'C:\Users\Jakinda\Documents\Python Scripts\10Academy\PharmaceuticalSalesPrediction\data\store.csv'
repo = r'C:\Users\Jakinda\Documents\Python Scripts\10Academy\PharmaceuticalSalesPrediction'
version = 'v3'

data_url = dvc.api.get_url(

    path = path,
    repo = repo,
    rev = version

)


In [4]:
store = pd.read_csv(data_url,sep=',')
store

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [5]:
path = r'C:\Users\Jakinda\Documents\Python Scripts\10Academy\PharmaceuticalSalesPrediction\data\train.csv'
repo = r'C:\Users\Jakinda\Documents\Python Scripts\10Academy\PharmaceuticalSalesPrediction'
version = 'v1'

data_url2 = dvc.api.get_url(

    path = path,
    repo = repo,
    rev = version

)

In [6]:
train = pd.read_csv(data_url2,sep=',')
train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [7]:
# merge the train with the stores set
train_store = pd.merge(left = train, right = store, how = 'inner', left_on = 'Store', right_on = 'Store')
print(train_store.shape)
#print(test_store.shape)

(1017209, 18)


In [8]:
train_store = train_store.drop(columns=['Customers','Store'])

target_name = "Sales"
target = train_store[target_name]

data = train_store.drop(columns=[target_name])

In [9]:
def feature_generation(data):
    data['Date'] = pd.to_datetime(data.Date)
    data['Month'] = data.Date.dt.month.to_list()
    data['Year'] = data.Date.dt.year.to_list()
    data['Day'] = data.Date.dt.day.to_list()
    data['WeekOfYear'] = data.Date.dt.weekofyear.to_list()
    data['DayOfWeek'] = data.Date.dt.dayofweek.to_list()
    data['weekday'] = 1        # Initialize the column with default value of 1
    data.loc[data['DayOfWeek'] == 5, 'weekday'] = 0
    data.loc[data['DayOfWeek'] == 6, 'weekday'] = 0
#         data = data.drop(['Date'], axis = 1)
        
    return data

In [10]:
data = train_dup=feature_generation(data)

In [11]:
data = data.drop(columns='Date')
data

,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Month,Year,Day,WeekOfYear,weekday
0,4,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,2015,31,31,1
1,3,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,2015,30,31,1
2,2,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,2015,29,31,1
3,1,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,2015,28,31,1
4,0,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,2015,27,31,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,5,1,0,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,2013,5,1,0
1017205,4,1,0,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,2013,4,1,1
1017206,3,1,0,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,2013,3,1,1
1017207,2,1,0,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",1,2013,2,1,1


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 19 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   DayOfWeek                  1017209 non-null  int64  
 1   Open                       1017209 non-null  int64  
 2   Promo                      1017209 non-null  int64  
 3   StateHoliday               1017209 non-null  object 
 4   SchoolHoliday              1017209 non-null  int64  
 5   StoreType                  1017209 non-null  object 
 6   Assortment                 1017209 non-null  object 
 7   CompetitionDistance        1014567 non-null  float64
 8   CompetitionOpenSinceMonth  693861 non-null   float64
 9   CompetitionOpenSinceYear   693861 non-null   float64
 10  Promo2                     1017209 non-null  int64  
 11  Promo2SinceWeek            509178 non-null   float64
 12  Promo2SinceYear            509178 non-null   float64
 13  PromoInterva

In [13]:
data["StateHoliday"] = data["StateHoliday"].map({0: 0, "0": 0, "a": 1, "b": 1, "c": 1})

In [14]:
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

In [15]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42)

In [16]:
data_train

,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Month,Year,Day,WeekOfYear,weekday
223369,6,0,0,0,0,c,a,4660.0,4.0,2013.0,0,NaN,NaN,NaN,3,2013,24,12,0
719159,5,1,0,0,1,d,c,9070.0,12.0,2003.0,0,NaN,NaN,NaN,1,2015,3,1,0
584261,1,1,0,0,0,a,c,610.0,12.0,2003.0,1,36.0,2013.0,"Mar,Jun,Sept,Dec",2,2013,26,9,1
699404,1,1,0,0,0,a,c,2630.0,9.0,2012.0,0,NaN,NaN,NaN,11,2013,12,46,1
650180,6,0,0,0,0,a,a,14810.0,6.0,2014.0,0,NaN,NaN,NaN,6,2015,21,25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,1,1,0,0,0,a,a,1460.0,4.0,2015.0,0,NaN,NaN,NaN,10,2014,28,44,1
365838,4,1,1,0,0,a,c,9200.0,10.0,2009.0,1,14.0,2012.0,"Jan,Apr,Jul,Oct",11,2013,8,45,1
131932,2,1,0,0,0,d,c,8050.0,10.0,1961.0,1,48.0,2012.0,"Jan,Apr,Jul,Oct",9,2014,24,39,1
671155,1,1,1,0,0,a,a,100.0,5.0,2007.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct",4,2013,9,15,1


In [42]:
cont_pipeline = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy = 'median')),
    ('power',PowerTransformer(method = 'yeo-johnson', standardize = False)),
    ('scaler',StandardScaler())
]
)

In [43]:
cat_pipeline = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy = 'most_frequent', fill_value = 'unknown')),
    ('onehot',OneHotEncoder(handle_unknown="ignore"))]
)

In [44]:
preprocessor = ColumnTransformer(
    transformers = [
        ('continuous', cont_pipeline, numerical_columns),
        ('categorical', cat_pipeline, categorical_columns)

    ]
)

In [21]:
def quick_eval(pipeline, X_train, y_train, X_test, y_test, verbose=True):
    """
    Quickly trains modeling pipeline and evaluates on test data.      Returns original model, training RMSE, and testing
    RMSE as a tuple.
    """
    
    pipeline.fit(X_train, y_train)
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)
    
    train_score = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_score = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    if verbose:
        print(f"Regression algorithm: {pipeline.named_steps['regressor'].__class__.__name__}")
        logger.info(f"Regression algorithm: {pipeline.named_steps['regressor'].__class__.__name__}")
        print(f"Train RMSE: {train_score}")
        logger.info(f"Train RMSE: {train_score}")
        print(f"Test RMSE: {test_score}")
        logger.info(f"Test RMSE: {test_score}")
        
    
    return pipeline.named_steps['regressor'], train_score, test_score
    
regressors = [
    DecisionTreeRegressor(),
    #LinearRegression(),
    #GradientBoostingRegressor(),
    #RandomForestRegressor()
]
for r in regressors:
    pipe = Pipeline(steps = [
        ('preprocessor', preprocessor),
        ('regressor', r)
    ])

    quick_eval(pipe, data_train, target_train, data_test, target_test)
    print()


Regression algorithm: DecisionTreeRegressor
Train RMSE: 105.94869640760602
Test RMSE: 1221.2776232342849

Regression algorithm: LinearRegression
Train RMSE: 2508.251324525623
Test RMSE: 2497.660038032195



In [26]:
DecisionTreeRegressor = DecisionTreeRegressor()

In [45]:
pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', DecisionTreeRegressor)
        ]
        )

In [46]:
pipe.fit(data_train,target_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('continuous',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('power',
                                                                   PowerTransformer(standardize=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['DayOfWeek', 'Open', 'Promo',
                                                   'StateHoliday',
                                                   'SchoolHoliday',
                                                   'CompetitionDistance',
                                                   'CompetitionOpenSinceMonth',
       

In [47]:
pipe.named_steps['preprocessor'].transformers_[1][1]\
   .named_steps['onehot'].get_feature_names(categorical_columns)

array(['StoreType_a', 'StoreType_b', 'StoreType_c', 'StoreType_d',
       'Assortment_a', 'Assortment_b', 'Assortment_c',
       'PromoInterval_Feb,May,Aug,Nov', 'PromoInterval_Jan,Apr,Jul,Oct',
       'PromoInterval_Mar,Jun,Sept,Dec'], dtype=object)

In [ ]:
len(r.feature_importances_)

26

In [ ]:
r == regressors[3]
importances = pd.DataFrame({
    'Feature': r.columns,
    'Importance': r.feature_importances_
})
importances = importances.sort_values(by='Importance', ascending=False)
importances = importances.set_index('Feature')
importances

In [ ]:
for r in regressors:
    if r == regressors[1]:
        print('hello')
    else:
        importance = r.feature_importances_
        

hello


Making a pipeline with randomForest

In [ ]:
model = make_pipeline(preprocessor, RandomForestRegressor())

In [ ]:
from sklearn import set_config
set_config(display='diagram')
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('continuous',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('powertransformer',
                                                                   PowerTransformer(standardize=False)),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['DayOfWeek', 'Open', 'Promo',
                                                   'StateHoliday',
                                                   'SchoolHoliday',
                                                   'CompetitionDistance',
                                                   'CompetitionOpenSinceMon...
                                                   'Promo2', 'Promo2SinceWeek',
                                                   'Promo2SinceYear', 'Month',
                                                   'Year', 'Day', 'WeekOfYear',
                                                   'weekday']),
                                                 ('categorical',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='unknown',
                                                                                 strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['StoreType', 'Assortment',
                                                   'PromoInterval'])])),
                ('randomforestregressor', RandomForestRegressor())])